# **Install package**

In [1]:
!pip install libauc

# **Import LibAUC**

In [2]:
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import ResNet18
from libauc.datasets import CIFAR10
from libauc.datasets import imbalance_generator 

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

# **Reproducibility**

In [3]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# **Imagedataset**

In [4]:
class ImageDataset(Dataset):
    def __init__(self, images, targets, mode='train'):
       self.images = images.astype(np.uint8)
       self.targets = targets
       self.mode = mode
       self.transform_train = transforms.Compose([
                              transforms.RandomCrop((30, 30), padding=None),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize((32, 32)),
                              transforms.ToTensor(),
                              ])
       self.transform_test = transforms.Compose([
                             transforms.Resize((32, 32)),
                             transforms.ToTensor(),
                              ])
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        if self.mode == 'train':
            image = self.transform_train(image)
        else:
            image = self.transform_test(image)
        return image, target

# **Paramaters**

In [5]:
# paramaters
SEED = 123
BATCH_SIZE = 64
imratio = 0.1
lr = 0.1
gamma = 300
weight_decay = 1e-4
margin = 1.0

# **Load datasets**

In [6]:
# dataloader 
(train_data, train_label), (test_data, test_label) = CIFAR10()
(train_images, train_labels) = imbalance_generator(train_data, train_label, imratio=imratio, shuffle=True, random_seed=SEED)
(test_images, test_labels) = imbalance_generator(test_data, test_label, is_balanced=True,  random_seed=SEED)

trainloader = torch.utils.data.DataLoader(ImageDataset(train_images, train_labels), batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True, drop_last=True)
testloader = torch.utils.data.DataLoader( ImageDataset(test_images, test_labels, mode='test'), batch_size=BATCH_SIZE, shuffle=False, num_workers=1,  pin_memory=True)

NUM_SAMPLES: [27777], POS:NEG: [2777 : 25000], POS_RATIO: 0.1000
NUM_SAMPLES: [10000], POS:NEG: [5000 : 5000], POS_RATIO: 0.5000


# **Load models & AUC Optimizer**

In [7]:
model = ResNet18(pretrained=False)
model = model.cuda()

Loss = AUCMLoss(imratio=imratio)
optimizer = PESG(model, a=Loss.a, b=Loss.b, alpha=Loss.alpha, imratio=imratio, lr=lr, gamma=gamma, margin=margin, weight_decay=weight_decay)

# **Training**

In [ ]:
for epoch in range(100):
    
     if epoch == 50 or epoch==75:
         optimizer.lr = optimizer.lr/10
         optimizer.update_regularizer()
   
     train_pred = []
     train_true = []
     model.train()    
     for data, targets in trainloader:
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
         loss = Loss(y_pred, targets)
         optimizer.zero_grad()
         loss.backward(retain_graph=True)
         optimizer.step()
        
         train_pred.append(y_pred.cpu().detach().numpy())
         train_true.append(targets.cpu().detach().numpy())

     train_true = np.concatenate(train_true)
     train_pred = np.concatenate(train_pred)
     train_auc = roc_auc_score(train_true, train_pred) 

     model.eval()
     test_pred = []
     test_true = [] 
     for j, data in enumerate(testloader):
         test_data, test_targets = data
         test_data = test_data.cuda()
         y_pred = model(test_data)
         test_pred.append(y_pred.cpu().detach().numpy())
         test_true.append(test_targets.numpy())
     test_true = np.concatenate(test_true)
     test_pred = np.concatenate(test_pred)
     val_auc =  roc_auc_score(test_true, test_pred) 
     model.train()
   
     # print results
     print("epoch: {}, train_loss: {:4f}, train_auc:{:4f}, test_auc:{:4f}, lr:{:4f}".format(epoch, loss.item(), train_auc, val_auc, optimizer.lr ))          

epoch: 0, train_loss: 0.081256, train_auc:0.600130, test_auc:0.620038, lr:0.100000
epoch: 1, train_loss: 0.060751, train_auc:0.664095, test_auc:0.683009, lr:0.100000
epoch: 2, train_loss: 0.091481, train_auc:0.702076, test_auc:0.711413, lr:0.100000
epoch: 3, train_loss: 0.092363, train_auc:0.729838, test_auc:0.730042, lr:0.100000
epoch: 4, train_loss: 0.065447, train_auc:0.753200, test_auc:0.721380, lr:0.100000
epoch: 5, train_loss: 0.071429, train_auc:0.773943, test_auc:0.754498, lr:0.100000
epoch: 6, train_loss: 0.048350, train_auc:0.784072, test_auc:0.737505, lr:0.100000
